# Comparing Peak WSE for historic events in Jacksonville

### Using the Coastal Hazard System (CHS) [https://chs.erdc.dren.mil/] for select points in Jacksonville, FL 
### Using data for CHS-SA_TH_SimB1HT_ADCIRC01_Peaks

CHS-SA | Phase II of CHS-SACS - North Carolina to South Florida
TH | Tropical Histoirical Storms
SimB1HT | Base plus 1 historical tide; No sea level change
ADCIRC01 | ADCIRC model
Post0 | No Post Processing

# The following map shows the locations of the points downloaded from CHS in this analysis.

In [4]:
import folium
import json
import os
# Show ADCIRC points on map using folium and a geojson
geojson_fn = "CHS ADCIRC Test Points.geojson" 
m = folium.Map(location=[30.3322, -81.6557], tiles="CartoDB Positron", zoom_start=10)

tooltip = folium.GeoJsonTooltip(
    fields=["Point"],
    aliases=["ADCIRC Point"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

folium.GeoJson(
    geojson_fn,
    tooltip=tooltip).add_to(m)

m

In [5]:

import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
import os
import glob

# Get the list of ADCIRC points from the geojson
with open(geojson_fn) as f:
    data = json.load(f)
    points = [feature["properties"]["Point"] for feature in data["features"]]



In [6]:
# Get the list of historical storms from the CHS data
# for each point, open the h5 file and get the list of storms.  Add to a list of storms and remove duplicates.
storms = []
for point in points:
    #  example hdf filename: "CHS-SA_TH_SimB1HT_ADCIRC01_Peaks\CHS-SA_TH_SimB1HT_Post0_SP17359_ADCIRC01_Peaks.h5"
    h5_fn = f"CHS-SA_TH_SimB1HT_ADCIRC01_Peaks/CHS-SA_TH_SimB1HT_Post0_SP{point}_ADCIRC01_Peaks.h5"
    with h5py.File(h5_fn, "r") as f:
        # storms.extend(list(f.keys()))
        # print the keys
        # print(point)
        # [print(storm.decode('utf-8')) for storm in f['Storm Name'][:]]
        # print(f['Storm Name'][:])
        # decode the storm names to string
        storms.extend([storm.decode('utf-8') for storm in f['Storm Name'][:]])

# view storm counts
from collections import Counter
a = dict(Counter(storms))
print (f'Each Storm Name Count should match number of Points = {len(points)}')
print(a)

# remove duplicates
storms = list(set(storms))


Each Storm Name Count should match number of Points = 18
{'Hugo': 18, 'Andrew': 18, 'Fran': 18, 'Frances': 18, 'Matthew': 18, 'Irma': 18, 'Florence': 18}


In [7]:
# For each point, get the peak WSE from the CHS HDF5 (.h5) data for each historical storm and save to a dataframe
for point in points:
    storms = []
    peak_wse = []
    h5_fn = f"CHS-SA_TH_SimB1HT_ADCIRC01_Peaks/CHS-SA_TH_SimB1HT_Post0_SP{point}_ADCIRC01_Peaks.h5"
    with h5py.File(h5_fn, "r") as f:
        # get the peak WSE for each storm
        storms.extend([storm.decode('utf-8') for storm in f['Storm Name'][:]])
        peak_wse = f["Water Elevation"][:]
        # print(peak_wse)
        # print(storms)

        # Add the peak WSE for each storm to the geojson
        for feature in data["features"]:
            if feature["properties"]["Point"] == point:
                for i, storm in enumerate(storms):
                    feature["properties"][f'{storm}_wse'] = peak_wse[i]

# Save the geojson with the peak WSE for each storm
with open("CHS ADCIRC Test Points with Peak WSE.geojson", "w") as f:
    json.dump(data, f)


# The Following table illustrates the CHS peak WSE data pulled for each point for multiple historic events.

In [8]:
# Geojson to pandas dataframe
import geopandas as gpd
gdf = gpd.read_file("CHS ADCIRC Test Points with Peak WSE.geojson")
gdf

,Point,Hugo_wse,Andrew_wse,Fran_wse,Frances_wse,Matthew_wse,Irma_wse,Florence_wse,geometry
0,17393,2.082267,2.173514,1.800435,1.730509,2.467037,2.692735,2.040012,POINT (-81.43500 30.52300)
1,17502,2.020779,2.117998,1.752374,1.707301,2.577516,2.704891,1.994340,POINT (-81.41100 30.46000)
2,17808,1.910270,2.009375,1.705437,1.685486,2.636136,2.660179,1.878746,POINT (-81.39700 30.36400)
3,18090,-99999.000000,-99999.000000,-99999.000000,-99999.000000,2.749486,2.712674,-99999.000000,POINT (-81.38580 30.27030)
4,18525,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,POINT (-81.46500 30.12600)
5,17386,1.355734,1.379408,1.373533,1.467996,2.287906,2.471192,1.467594,POINT (-81.49400 30.52500)
6,18172,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,POINT (-81.50500 30.24500)
7,18413,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,POINT (-81.61710 30.15870)
8,17804,0.966477,0.841390,0.958371,1.207413,1.582741,2.046001,1.034894,POINT (-81.61300 30.36500)
9,17359,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,POINT (-81.62400 30.53800)


In [9]:
# Drop points with noData
gdf = gdf[gdf.Hugo_wse != -99999.0]

# Create geojson with onlly valid data points
gdf.to_file("CHS ADCIRC Test Points with Peak WSE (no missing data).geojson", driver='GeoJSON')

gdf

,Point,Hugo_wse,Andrew_wse,Fran_wse,Frances_wse,Matthew_wse,Irma_wse,Florence_wse,geometry
0,17393,2.082267,2.173514,1.800435,1.730509,2.467037,2.692735,2.040012,POINT (-81.43500 30.52300)
1,17502,2.020779,2.117998,1.752374,1.707301,2.577516,2.704891,1.994340,POINT (-81.41100 30.46000)
2,17808,1.910270,2.009375,1.705437,1.685486,2.636136,2.660179,1.878746,POINT (-81.39700 30.36400)
5,17386,1.355734,1.379408,1.373533,1.467996,2.287906,2.471192,1.467594,POINT (-81.49400 30.52500)
8,17804,0.966477,0.841390,0.958371,1.207413,1.582741,2.046001,1.034894,POINT (-81.61300 30.36500)
15,18084,1.167256,1.048769,1.152675,1.225181,1.978545,1.870136,1.217775,POINT (-81.40920 30.27200)
17,18107,1.172820,1.053388,1.157783,1.274820,1.994416,1.953627,1.220706,POINT (-81.42400 30.26600)


In [10]:
# Create a folium map with the geojson containing the peak WSE for each storm.
# The map will have a slider to show the peak WSE for each storm.
# The map will have a legend to show the color scale for the peak WSE.
import folium
from folium.features import DivIcon


geojson_fn = "CHS ADCIRC Test Points with Peak WSE (no missing data).geojson"
fields=[
                "Point",
				"Hugo_wse",
				"Andrew_wse",
				"Fran_wse",
				"Frances_wse",
				"Matthew_wse",
				"Irma_wse",
				"Florence_wse"
            ]
m = folium.Map(location=[30.3322, -81.6557], tiles="CartoDB Positron", zoom_start=10)

tooltip = folium.GeoJsonTooltip(
    fields=fields,
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
        fill-opacity: 0.1;
    """,
    max_width=800,
    opacity=0.5,
)

popup = folium.GeoJsonPopup(
    fields=fields,
    # aliases=["ADCIRC Point"],
    localize=True,
    labels=True,
    style="""
        border: 0px solid black;
        border-radius: 1px;
        box-shadow: 1px;
    """,
    opacity=0.5,
    
)

folium.GeoJson(
    geojson_fn,
    tooltip=tooltip,
    popup=popup).add_to(m)

m
